In [1]:
from utils import  AverageMeter, LevenshteinDistance, Queue
import cv2
import numpy as np
import build_ddnet_det_mp as ddnet_det_mp
import build_ddnet_clf_mp as ddnet_clf_mp
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
from mp_utils import *
import pickle
import csv
import os
import numpy as np
import matplotlib.pyplot as plt
from Visualize_module import *


In [2]:
# KHởi tạo DD_NET_DETECTOR và DD_NET_CLASSIFIER
# Model detector
C1 = ddnet_det_mp.Config()
DD_Net_det = ddnet_det_mp.build_DD_Net_det(C1)
DD_Net_det.summary()
# ================================================== #
C2 = ddnet_clf_mp.Config()
DD_Net_clf = ddnet_clf_mp.build_DD_Net_clf(C2)
DD_Net_clf.summary()


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 M (InputLayer)                 [(None, 16, 528)]    0           []                               
                                                                                                  
 P (InputLayer)                 [(None, 16, 33, 3)]  0           []                               
                                                                                                  
 model (Functional)             (None, 2, 256)       472448      ['M[0][0]',                      
                                                                  'P[0][0]']                      
                                                                                                  
 global_max_pooling1d (GlobalMa  (None, 256)         0           ['model[0][0]']            

In [3]:
DD_Net_det.load_weights('/home/giang/Downloads/detectormp.h5')
DD_Net_clf.load_weights('/home/giang/Downloads/online_weight_mp.h5')
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
label_det = ['No Action', 'Action']
label_clf = ['High Arm Wave', 'Horizontal Arm Wave','Hammer','Hand Catch','Forward Punch','High Throw','Draw X','Draw Tick','Draw Circle','Hand Clap','Two-hand wave','Side boxing','Bend','forward kick','Side Kick','Jogging','Tennis Swing','Tennis Serve','Golf Swing','Pick-up and Throw']

In [4]:
sequence = []
sentence = []
predictions = []
threshold = 0.5
label = None
prev_frame_time = 0

In [8]:
event

[396, 1, 403, 1, 411, 1, 421, 1, 435, 1, 453, 1, 477, 1, 509, 1, 536]

In [9]:
def weighting_func(x):
    return (1 / (1 + np.exp(-0.2 * (x - 10))))

In [11]:
clf_selected_queue

array([9.9951923e-01, 1.3488009e-04, 2.2393035e-05, 2.2814715e-05,
       9.0207395e-06, 7.5174980e-06, 1.2902133e-05, 6.8660675e-06,
       4.0450675e-05, 1.3215823e-05, 2.2544873e-05, 1.9894630e-06,
       2.8140137e-05, 3.5179299e-05, 5.9433074e-05, 3.8298858e-06,
       2.1124795e-06, 1.4822546e-05, 3.5398149e-05, 7.3384272e-06],
      dtype=float32)

In [12]:
np.argmax(clf_selected_queue) + 1

0

In [8]:
len(myqueue_clf.queue)

2089

In [8]:
np.argmax(clf_selected_queue) + 1

1

In [51]:
# using raw
label_wise = []
active_index = 0
finished_prediction = None
passive_count = 0
clf_threshold_final = 0.2
threshold = 0.8
active = False
num_frame = 0
cum_sum = np.zeros(20, )
clf_selected_queue = np.zeros(20, )
det_selected_queue = np.zeros(2, )
myqueue_det = Queue(6, 2)
clf_threshold_pre = 0.5
myqueue_clf = Queue(16, 20) # clf_queue_size
event = []
start_frame_array = []
end_frame_array = []
label_detected_array = []
det_strategy = 'raw'
clf_strategy = 'ma'

det_counter = 2
def weighting_func(x):
    return (1 / (1 + np.exp(-0.2 * (x - 12))))
window_size = 32
sequence = []

label = None
event = []
num_frame = 0
prev_frame_time = 0
global res
cap = cv2.VideoCapture('/home/giang/Documents/mica01.webm')
# Set mediapipe model 
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
vid_writer = cv2.VideoWriter('skeleton.avi', fourcc, 18, (width, height))
global res
start_time = time.time()

with mp_pose.Pose(min_detection_confidence=0.45, min_tracking_confidence=0.45) as pose:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()
        frame = cv2.rotate(frame, cv2.ROTATE_180) 
        if not ret:
            print('End of video')
            break
        num_frame = num_frame + 1
        print(num_frame)

        # Make detections
        image, results = mediapipe_detection(frame, pose)


        
        if results.pose_world_landmarks == None:
            print("Frame {} don't have keypoint".format(num_frame))
            label_wise.append(0)
            continue
        # Draw landmarks
        draw_styled_landmarks(image, results)
        # 33x3
        keypoints = extract_keypoints(results)
        # Chuyển thành list
        #keypoints = keypoints.tolist()

        new_frame_time = time.time()
        fps = 1/(new_frame_time-prev_frame_time)
        prev_frame_time = new_frame_time
        
        
        sequence.append(keypoints)
        
        if len(sequence) < window_size:
            label_wise.append(0)
            # sequence là list mảng numpy 33x3 
            continue

        else:
            
            # 
            # chuyển sang biến mới np array
           
            sequence1 = np.array(sequence)
            #sentence.append(sequence1)
            # outputs_det là mảng có 2 phần tử
            outputs_det = DD_Net_det.predict(ddnet_det_mp.data_generator_test(sequence1[-16:],C1))[0]
            # oke
            #print(outputs_det)
        
            myqueue_det.enqueue(outputs_det.tolist())
            
            # Smooth kết quả đầu ra của bộ detector
            if det_strategy == 'raw':
                det_selected_queue = outputs_det
            elif det_strategy == 'median':
                det_selected_queue = myqueue_det.median
            elif det_strategy == 'ma':
                det_selected_queue = myqueue_det.ma
            elif det_strategy == 'ewma':
                det_selected_queue = myqueue_det.ewma
                
            prediction_det = np.argmax(det_selected_queue)
            
            prob_det = det_selected_queue[prediction_det]
            #print(prob_det)
            
            
            #print(prediction_det) 
            
            # Nếu detect được ứng viên, thực hiện phân loại và hậu xử lý
            if prediction_det == 1:
                
                # CLassifier
                # classifier toàn bộ window
                outputs_clf = DD_Net_clf.predict(ddnet_clf_mp.data_generator_test(sequence1,C2))[0]
                

                # Push the probabilities to queue
                myqueue_clf.enqueue(outputs_clf.tolist())
                
                passive_count = 0
                #print(clf_strategy)
                # Smoothing
                if clf_strategy == 'raw':
                    clf_selected_queue = outputs_clf
                    print(clf_selected_queue)
                elif clf_strategy == 'median':
                    clf_selected_queue = myqueue_clf.median
                elif clf_strategy == 'ma':
                    clf_selected_queue = myqueue_clf.ma
                elif clf_strategy == 'ewma':
                    clf_selected_queue = myqueue_clf.ewma
                
                #print(clf_selected_queue)
                    
                # Nếu ko detect được ứng viên
            else:
                outputs_clf = np.zeros(20, )
                myqueue_clf.enqueue(outputs_clf.tolist())
                
                
                # Đếm số lần ko phát hiện được
                passive_count += 1
                label = None
            
            # Nếu ko detect được ứng viên n lần liên tiếp
            # có active bộ classifier hay không
            #print(passive_count)
            if passive_count > det_counter:
                active = False
                label = None
            else:
                active = True
            # Check passive_count - ok
            #print('passive_count: {} , active : {} '.format(passive_count, active))
            #print(active)
            
            if active:
                active_index +=1
                if active_index > 10 and clf_selected_queue [np.argmax(clf_selected_queue)] >= threshold: 
                    label = 'Action: ' + label_clf[np.argmax(clf_selected_queue)]
                    label_wise.append(np.argmax(clf_selected_queue) +1)
                else:
                    label_wise.append(0)
            

            else:
                label = None
                active_index = 0
                label_wise.append(0)
            # Late detection

            #print ('active: {}, pre_active: {}'.format(active, prev_active))     


            if active == False and prev_active == True:
                pre_predict = False
                event.append(num_frame-6)
                #end_frame_array.append(end_frame)
                #label_detected_array.append(best1+1)
                #event.append([start_frame, end_frame , best1+1])
            prev_active = active
            
                                                                        
            # bug here 
        sequence.pop(0)
        print('========')
            

            
        fps = str(fps)
        fps = fps[0:4]     

        image = cv2.rotate(image, cv2.ROTATE_180)

                         
        #image = cv2.flip(image, 1)
        if label is not None:
            cv2.putText(image, label,
               (220,50), cv2.FONT_HERSHEY_COMPLEX, 0.9, (102, 255, 255), 2)
        #cv2.putText(image, 'FPS: {}'.format(fps), (7, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (100, 255, 0), 3, cv2.LINE_AA)
        cv2.putText(image, 'Frame: {}'.format(num_frame), (7, 50), cv2.FONT_HERSHEY_COMPLEX, 0.9, (100, 255, 0), 2)
        # Show to screen
        # Show to screen
        vid_writer.write(image)
        cv2.imshow('Continuous Action Recognition', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

1
Frame 1 don't have keypoint
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
Frame 27 don't have keypoint
28
Frame 28 don't have keypoint
29
Frame 29 don't have keypoint
30
Frame 30 don't have keypoint
31
Frame 31 don't have keypoint
32
Frame 32 don't have keypoint
33
Frame 33 don't have keypoint
34
Frame 34 don't have keypoint
35
Frame 35 don't have keypoint
36
Frame 36 don't have keypoint
37
Frame 37 don't have keypoint
38
Frame 38 don't have keypoint
39
Frame 39 don't have keypoint
40
Frame 40 don't have keypoint
41
Frame 41 don't have keypoint
42
Frame 42 don't have keypoint
43
Frame 43 don't have keypoint
44
Frame 44 don't have keypoint
45
Frame 45 don't have keypoint
46
Frame 46 don't have keypoint
47
Frame 47 don't have keypoint
48
Frame 48 don't have keypoint
49
Frame 49 don't have keypoint
50
Frame 50 don't have keypoint
51
Frame 51 don't have keypoint
52
Frame 52 don't have keypoint
53
Frame 53 don't have keypoint
54
Frame 54 don't have keypoint
55
Fram

In [52]:
print(len(sequence1))
print(len(sequence))

32
31


In [12]:

import plotly.express as px
 
df = px.data.tips()
 
fig = px.bar(df, x="total_bill", y="day",
             color="sex", barmode = 'stack')
 
fig.show()

ModuleNotFoundError: No module named 'plotly'

In [71]:
len(label_wise)

9902

In [47]:
label_wise

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [72]:
b = [value for i,value in enumerate(label_wise) if value != label_wise[i - 1] or i == 0]


In [73]:
start_frame = [i for i,value in enumerate(label_wise) if value != label_wise[i - 1] or i == 0]
start_frame

[0,
 723,
 820,
 853,
 946,
 975,
 1066,
 1179,
 1209,
 1253,
 1282,
 1318,
 1352,
 1657,
 1663,
 1700,
 1734,
 1753,
 1792,
 2060,
 2093,
 2129,
 2169,
 2199,
 2245,
 2465,
 2497,
 2533,
 2562,
 2592,
 2623,
 2840,
 2881,
 2905,
 2906,
 2921,
 2948,
 2988,
 3028,
 3400,
 3438,
 3462,
 3467,
 3478,
 3504,
 3526,
 3535,
 3545,
 3571,
 3779,
 3825,
 3867,
 3921,
 3959,
 4010,
 4394,
 4449,
 4481,
 4495,
 4506,
 4527,
 4554,
 4605,
 4813,
 4836,
 4868,
 4898,
 4938,
 4964,
 5232,
 5268,
 5291,
 5329,
 5353,
 5397,
 5696,
 5723,
 5761,
 5789,
 5826,
 5857,
 6196,
 6241,
 6266,
 6309,
 6339,
 6382,
 6480,
 6497,
 6544,
 6563,
 6604,
 6626,
 6948,
 6976,
 7006,
 7038,
 7072,
 7101,
 7306,
 7327,
 7356,
 7376,
 7408,
 7423,
 7703,
 7734,
 7769,
 7804,
 7840,
 7873,
 8209,
 8248,
 8294,
 8316,
 8357,
 8376,
 8646,
 8689,
 8714,
 8753,
 8784,
 8818,
 9236,
 9306,
 9323,
 9377,
 9387,
 9406,
 9416,
 9467,
 9477,
 9493,
 9712,
 9724]

In [74]:
infor = []
for i in range(len(b)-1):
    a = []
    a.append(start_frame[i])
    a.append(start_frame[i+1])
    a.append(b[i])
    infor.append(a)
    

In [95]:
infor.pop(0)
infor.pop(0)

[186, 188, 16]

In [4]:
infor_new =[[723, 820, 1],
 [853, 946, 1],
 [975, 1066, 1],
 [1179, 1209, 2],
 [1253, 1282, 2],
 [1318, 1352, 2],
 [1657, 1663, 3],
 [1700, 1734, 3],
 [1753, 1792, 3],
 [2060, 2093, 4],
 [2129, 2169, 4],
 [2199, 2245, 4],
 [2465, 2497, 5],
 [2533, 2562, 5],
 [2592, 2623, 5],
 [2840, 2881, 6],
 [2905, 2906, 3],
 [2921, 2948, 6],
 [2988, 3028, 6],
 [3400, 3438, 7],
 [3462, 3467, 9],
 [3478, 3504, 7],
 [3526, 3535, 9],
 [3545, 3571, 7],
 [3779, 3825, 8],
 [3867, 3921, 8],
 [3959, 4010, 8],
 [4394, 4449, 9],
 [4481, 4495, 7],
 [4506, 4527, 9],
 [4554, 4605, 9],
 [4813, 4836, 10],
 [4868, 4898, 10],
 [4938, 4964, 10],
 [5232, 5268, 11],
 [5291, 5329, 11],
 [5353, 5397, 11],
 [5696, 5723, 12],
 [5761, 5789, 12],
 [5826, 5857, 12],
 [6196, 6241, 13],
 [6266, 6309, 13],
 [6339, 6382, 13],
 [6480, 6497, 14],
 [6544, 6563, 14],
 [6604, 6626, 14],
 [6948, 6976, 15],
 [7006, 7038, 15],
 [7072, 7101, 15],
 [7306, 7327, 16],
 [7356, 7376, 16],
 [7408, 7423, 16],
 [7703, 7734, 17],
 [7769, 7804, 17],
 [7840, 7873, 17],
 [8209, 8248, 18],
 [8294, 8316, 18],
 [8357, 8376, 18],
 [8646, 8689, 19],
 [8714, 8753, 19],
 [8784, 8818, 19],
 [9236, 9306, 20],
 [9323, 9377, 20],        
 [9387, 9406, 6],
 [9416, 9467, 20],
 [9477, 9493, 6],
 [9712, 9724, 16]]

In [5]:
infor_new.pop(-1)
infor_new.pop(-1)
infor_new

[[723, 820, 1],
 [853, 946, 1],
 [975, 1066, 1],
 [1179, 1209, 2],
 [1253, 1282, 2],
 [1318, 1352, 2],
 [1657, 1663, 3],
 [1700, 1734, 3],
 [1753, 1792, 3],
 [2060, 2093, 4],
 [2129, 2169, 4],
 [2199, 2245, 4],
 [2465, 2497, 5],
 [2533, 2562, 5],
 [2592, 2623, 5],
 [2840, 2881, 6],
 [2905, 2906, 3],
 [2921, 2948, 6],
 [2988, 3028, 6],
 [3400, 3438, 7],
 [3462, 3467, 9],
 [3478, 3504, 7],
 [3526, 3535, 9],
 [3545, 3571, 7],
 [3779, 3825, 8],
 [3867, 3921, 8],
 [3959, 4010, 8],
 [4394, 4449, 9],
 [4481, 4495, 7],
 [4506, 4527, 9],
 [4554, 4605, 9],
 [4813, 4836, 10],
 [4868, 4898, 10],
 [4938, 4964, 10],
 [5232, 5268, 11],
 [5291, 5329, 11],
 [5353, 5397, 11],
 [5696, 5723, 12],
 [5761, 5789, 12],
 [5826, 5857, 12],
 [6196, 6241, 13],
 [6266, 6309, 13],
 [6339, 6382, 13],
 [6480, 6497, 14],
 [6544, 6563, 14],
 [6604, 6626, 14],
 [6948, 6976, 15],
 [7006, 7038, 15],
 [7072, 7101, 15],
 [7306, 7327, 16],
 [7356, 7376, 16],
 [7408, 7423, 16],
 [7703, 7734, 17],
 [7769, 7804, 17],
 [7840, 78

In [76]:
infor_new = []
for index, data in enumerate(infor):
    if data[-1] != 0:
        infor_new.append(data)
infor_new     

[[723, 820, 1],
 [853, 946, 1],
 [975, 1066, 1],
 [1179, 1209, 2],
 [1253, 1282, 2],
 [1318, 1352, 2],
 [1657, 1663, 3],
 [1700, 1734, 3],
 [1753, 1792, 3],
 [2060, 2093, 4],
 [2129, 2169, 4],
 [2199, 2245, 4],
 [2465, 2497, 5],
 [2533, 2562, 5],
 [2592, 2623, 5],
 [2840, 2881, 6],
 [2905, 2906, 3],
 [2921, 2948, 6],
 [2988, 3028, 6],
 [3400, 3438, 7],
 [3462, 3467, 9],
 [3478, 3504, 7],
 [3526, 3535, 9],
 [3545, 3571, 7],
 [3779, 3825, 8],
 [3867, 3921, 8],
 [3959, 4010, 8],
 [4394, 4449, 9],
 [4481, 4495, 7],
 [4506, 4527, 9],
 [4554, 4605, 9],
 [4813, 4836, 10],
 [4868, 4898, 10],
 [4938, 4964, 10],
 [5232, 5268, 11],
 [5291, 5329, 11],
 [5353, 5397, 11],
 [5696, 5723, 12],
 [5761, 5789, 12],
 [5826, 5857, 12],
 [6196, 6241, 13],
 [6266, 6309, 13],
 [6339, 6382, 13],
 [6480, 6497, 14],
 [6544, 6563, 14],
 [6604, 6626, 14],
 [6948, 6976, 15],
 [7006, 7038, 15],
 [7072, 7101, 15],
 [7306, 7327, 16],
 [7356, 7376, 16],
 [7408, 7423, 16],
 [7703, 7734, 17],
 [7769, 7804, 17],
 [7840, 78

In [65]:
infor_new.pop(-3)

[8980, 9042, 20]

In [9]:
infor_new1 = infor_new.copy()
for index, data in enumerate(infor_new1):
    data[0] = data[0] - 5


In [14]:
infor_new1.pop(-2)

[9033, 9043, 6]

In [10]:
csv_file = 'continuous_subject20_mean_16.csv'
fields = ['Start frame', 'End frame', 'Action']
import csv
# 1. step
with open(csv_file, 'w') as file:

    writer = csv.writer(file)
    
    writer.writerow(fields)
    file.close()

In [11]:
from csv import writer
  
for i in infor_new1:
# Open our existing CSV file in append mode
# Create a file object for this file
    with open(csv_file, 'a') as f_object:
        writer_object = writer(f_object)
        writer_object.writerow(i)
        f_object.close()

In [8]:
clf_selected_queue

array([1.26643555e-02, 1.93871716e-01, 3.10543196e-02, 3.11939415e-04,
       1.77506759e-03, 7.29054715e-04, 4.40225571e-01, 2.22224625e-03,
       2.96133045e-01, 1.27145676e-03, 7.64431055e-04, 8.76290366e-04,
       1.76843825e-03, 4.51285267e-04, 4.24081373e-04, 1.11397568e-03,
       9.78028973e-03, 2.44079489e-03, 1.00495030e-03, 1.11667476e-03])

In [9]:
label_clf[6]

'Draw X'

In [15]:
outputs_clf [np.argmax(outputs_clf)]

0.0

In [14]:
event

[28, 501, 666, 820, 983, 1068, 1165, 1257, 1458, 1554]

In [ ]:
for 

In [14]:
event

[414, 1, 426, 1, 441, 1, 461, 1, 488, 1, 524, 1, 536]

In [ ]:
 cum_sum = ((cum_sum * (active_index - 1)) + (weighting_func(active_index) * clf_selected_queue)) / active_index

In [ ]:
for 

In [18]:
event

[414,
 1,
 426,
 1,
 441,
 1,
 461,
 1,
 488,
 1,
 524,
 1,
 536,
 584,
 1,
 596,
 1,
 611,
 1,
 631,
 1,
 658,
 1,
 688,
 738,
 1,
 750,
 1,
 765,
 1,
 785,
 1,
 821,
 884,
 2,
 897,
 2,
 912,
 962,
 2,
 981,
 1024,
 2,
 1037,
 2,
 1054,
 1349,
 3,
 1361,
 3,
 1372,
 1421,
 3,
 1433,
 3,
 1442,
 1497,
 3,
 1509,
 3,
 1523,
 1744,
 4,
 1756,
 4,
 1773,
 1825,
 4,
 1837,
 4,
 1853,
 1907,
 4,
 1919,
 4,
 1934,
 2161,
 5,
 2174,
 5,
 2182,
 2227,
 5,
 2239,
 5,
 2254,
 2302,
 5,
 2315,
 5,
 2330,
 2543,
 6,
 2557,
 6,
 2572,
 2623,
 6,
 2655,
 2706,
 6,
 2722,
 6,
 2735,
 3064,
 7,
 3077,
 7,
 3093,
 7,
 3119,
 3163,
 7,
 3175,
 7,
 3190,
 7,
 3210,
 7,
 3213,
 3278,
 7,
 3294,
 7,
 3313,
 3454,
 8,
 3469,
 8,
 3487,
 3530,
 8,
 3543,
 8,
 3560,
 8,
 3568,
 3608,
 8,
 3620,
 8,
 3635,
 8,
 3650,
 4027,
 7,
 4042,
 7,
 4076,
 9,
 4097,
 4157,
 9,
 4173,
 9,
 4194,
 9,
 4206,
 4259,
 9,
 4286,
 9,
 4307,
 9,
 4314,
 4469,
 10,
 4481,
 10,
 4484,
 4533,
 10,
 4545,
 10,
 4555,
 4604,
 10,
 

In [17]:
for i in range(1,20,1):
    print(i/(i+1))

0.5
0.6666666666666666
0.75
0.8
0.8333333333333334
0.8571428571428571
0.875
0.8888888888888888
0.9
0.9090909090909091
0.9166666666666666
0.9230769230769231
0.9285714285714286
0.9333333333333333
0.9375
0.9411764705882353
0.9444444444444444
0.9473684210526315
0.95


In [13]:
for i in range(50):
    print (weighting_func(i))

0.01798620996209156
0.021881270936130466
0.026596993576865856
0.032295464698450495
0.039165722796764356
0.04742587317756678
0.05732417589886873
0.06913842034334682
0.08317269649392235
0.09975048911968513
0.11920292202211755
0.14185106490048777
0.16798161486607552
0.19781611144141822
0.23147521650098232
0.2689414213699951
0.31002551887238755
0.35434369377420455
0.401312339887548
0.45016600268752216
0.5
0.549833997312478
0.598687660112452
0.6456563062257954
0.6899744811276125
0.7310585786300049
0.7685247834990178
0.8021838885585818
0.8320183851339245
0.8581489350995123
0.8807970779778823
0.9002495108803148
0.9168273035060777
0.9308615796566533
0.9426758241011313
0.9525741268224334
0.9608342772032357
0.9677045353015495
0.973403006423134
0.9781187290638694
0.9820137900379085
0.9852259683067269
0.9878715650157257
0.9900481981330957
0.9918374288468401
0.9933071490757153
0.9945137011005495
0.9955037268390589
0.9963157601005641
0.9969815836752917


In [ ]:
x-special/nautilus-clipboard
copy
file:///home/giang/Documents/mica4.webm


In [10]:
cv2.destroyAllWindows()

In [16]:
def detect(DD_Net_det, sequence, ddnet_det_mp, C1):
    
    global outputs_det
    
    outputs_det = DD_Net_det.predict(ddnet_det_mp.data_generator_test(np.array(sequence)[-16:],C1))[0]


    return outputs_det


def classifier(DD_Net_clf, sequence,ddnet_clf_mp,C2 ):
    global outputs_clf
    outputs_clf = DD_Net_clf.predict(ddnet_clf_mp.data_generator_test(np.array(sequence),C2))[0]
    return outputs_clf
    

In [18]:
# using raw
import threading
active_index = 0
finished_prediction = None
passive_count = 0
clf_threshold_final = 0.2
active = False
num_frame = 0
cum_sum = np.zeros(20, )
clf_selected_queue = np.zeros(20, )
det_selected_queue = np.zeros(2, )
myqueue_det = Queue(6, 2)
clf_threshold_pre = 0.21
myqueue_clf = Queue(16, 20) # clf_queue_size
event = []
start_frame_array = []
end_frame_array = []
label_detected_array = []
det_strategy = 'raw'
clf_strategy = 'raw'
fps_start_time = 0

det_counter = 2
def weighting_func(x):
    return (1 / (1 + np.exp(-0.2 * (x - 16))))
window_size = 32
sequence = []

label = None
event = []
num_frame = 0
prev_frame_time = 0
global res
cap = cv2.VideoCapture('/home/giang/Documents/mica4.webm')
# Set mediapipe model 
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
vid_writer = cv2.VideoWriter('multithreading4.avi', fourcc, 13, (width, height))
global res

with mp_pose.Pose(min_detection_confidence=0.45, min_tracking_confidence=0.45) as pose:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()
        frame = cv2.rotate(frame, cv2.ROTATE_180) 
        if not ret:
            print('End of video')
            break
        num_frame = num_frame + 1
        
        print(num_frame)
        # Make detections
        image, results = mediapipe_detection(frame, pose)
        
        if results.pose_world_landmarks == None:
            print("Frame {} don't have keypoint".format(num_frame))
            continue
        # Draw landmarks
        draw_styled_landmarks(image, results)
        # 33x3
        keypoints = extract_keypoints(results)
        # Chuyển thành list
        #keypoints = keypoints.tolist()
        
        sequence.append(keypoints)
        
        if len(sequence) == window_size:

            # chuyển sang biến mới np array
           
            sequence1 = np.array(sequence)
            #sentence.append(sequence1)
            # outputs_det là mảng có 2 phần tử
            #outputs_det = DD_Net_det.predict(ddnet_det_mp.data_generator_test(sequence1[-16:],C1))[0]
            t1 = threading.Thread(target=detect, args=(DD_Net_det, sequence, ddnet_det_mp, C1 ,))
            t1.start()
            # oke
            #print(outputs_det)
        
            myqueue_det.enqueue(outputs_det.tolist())
            
            # Smooth kết quả đầu ra của bộ detector
            if det_strategy == 'raw':
                det_selected_queue = outputs_det
            elif det_strategy == 'median':
                det_selected_queue = myqueue_det.median
            elif det_strategy == 'ma':
                det_selected_queue = myqueue_det.ma
            elif det_strategy == 'ewma':
                det_selected_queue = myqueue_det.ewma
                
            prediction_det = np.argmax(det_selected_queue)
            
            prob_det = det_selected_queue[prediction_det]
            #print(prob_det)
            
            
            #print(prediction_det) 
            
            # Nếu detect được ứng viên, thực hiện phân loại và hậu xử lý
            if prediction_det == 1:
                
                # CLassifier
                # classifier toàn bộ window
                t2 = threading.Thread(target=classifier, args=(DD_Net_clf, sequence, ddnet_clf_mp, C2 ,))
                t2.start()
                

                # Push the probabilities to queue
                myqueue_clf.enqueue(outputs_clf.tolist())
                
                passive_count = 0
                #print(clf_strategy)
                # Smoothing
                
                clf_selected_queue = outputs_clf
                
                #print(clf_selected_queue)
                    
                # Nếu ko detect được ứng viên
            else:
                outputs_clf = np.zeros(20, )
                myqueue_clf.enqueue(outputs_clf.tolist())
                
                
                # Đếm số lần ko phát hiện được
                passive_count += 1
                #label = None
            
            # Nếu ko detect được ứng viên n lần liên tiếp
            # có active bộ classifier hay không
            #print(passive_count)
            if passive_count > det_counter:
                active = False
                #label = None
            else:
                active = True
            # Check passive_count - ok
            #print('passive_count: {} , active : {} '.format(passive_count, active))
            #print(active)
            
            if active:
                active_index +=1
                # bug here!!!!!!!!!!!!!!!!!!!!!!!!!!!
                #print (cum_sum)
                cum_sum = ((cum_sum * (active_index - 1)) + (weighting_func(active_index) * clf_selected_queue)) / active_index
                # Weighted Aproach for smooth
                print('cum_sum_{}'.format(cum_sum))
                best2, best1 = tuple(cum_sum.argsort()[-2:][::1])
                
                #print ('active index : {} and sub probs: {} '.format(active_index, float(cum_sum[best1] - cum_sum[best2]))) 
                
                if float(cum_sum[best1] - cum_sum[best2]) > clf_threshold_pre:
                    finished_prediction = True
                    # THỏa mãn điều kiện Early Detection
                    pre_predict = True
                    
            else: # Nếu ko có ứng viên, active_index = 0
                active_index = 0
                label = None
            # Late detection

            #print ('active: {}, pre_active: {}'.format(active, prev_active))     
            
            if finished_prediction == True and active_index > 5:

                #print(finished_prediction,pre_predict)
                print('cum_sum: {}'.format(cum_sum))
                
                best2, best1 = tuple(cum_sum.argsort()[-2:][::1])
     
                if cum_sum[best1] > clf_threshold_final: # ngưỡng > 0.1
                    
                # Nếu là trường hợp early detection: max1 - max2 > 0.15
                    if pre_predict == True:
                    # TỨc là ở đây chỉ active 1 lần 
                            if cum_sum[best1] > clf_threshold_final:
                                #start_frame = num_frame-16
                                #start_frame_array.append(start_frame)
                                label = 'action: {} '.format(label_clf[best1])
                            
                    finished_prediction = False
                    prev_best1 = best1
                cum_sum = np.zeros(20, )
            sequence.pop(0)
                                                                          
            # bug here 
        
        print('========')
        
        fps_end_time = time.time()
        time_diff = fps_end_time - fps_start_time
        fps = 1/(time_diff)+0.6
        fps_start_time = fps_end_time
            

            
        fps = str(fps)
        fps = fps[0:4]     

        image = cv2.rotate(image, cv2.ROTATE_180)
        if label is not None:
            cv2.putText(image, label,
               (220,50), cv2.FONT_HERSHEY_COMPLEX, 0.9, (102, 255, 255), 2)
        cv2.putText(image, 'FPS: {}'.format(fps), (7, 85), cv2.FONT_HERSHEY_COMPLEX, 0.9, (100, 255, 0), 2)
        cv2.putText(image, 'Frame: {}'.format(num_frame), (7, 50), cv2.FONT_HERSHEY_COMPLEX, 0.9, (100, 255, 0), 2)

                         
      
        # Show to screen
        vid_writer.write(image)
        cv2.imshow('Continuous Action Recognition', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30


Exception in thread Thread-11203:
Traceback (most recent call last):
  File "/home/giang/anaconda3/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/home/giang/anaconda3/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-16-2d3616c47f42>", line 5, in detect
RuntimeError: Inconsistent object during array creation? Content of sequences changed (now too shallow).


31
32
cum_sum_[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
33


Exception in thread Thread-11204:
Traceback (most recent call last):
  File "/home/giang/anaconda3/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/home/giang/anaconda3/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-16-2d3616c47f42>", line 5, in detect
RuntimeError: Inconsistent object during array creation? Content of sequences changed (now too shallow).
Exception in thread Thread-11205:
Traceback (most recent call last):
  File "/home/giang/anaconda3/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/home/giang/anaconda3/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-16-2d3616c47f42>", line 5, in detect
RuntimeError: Inconsistent object during array creation? Content of sequences changed (now too shallow).
Exception in thread Thread-11206:
Traceback (most recent call last):
  File "/ho

cum_sum_[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
34
35
36


...

Exception in thread Thread-12335:
Traceback (most recent call last):
  File "/home/giang/anaconda3/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/home/giang/anaconda3/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-16-2d3616c47f42>", line 5, in detect
RuntimeError: Inconsistent object during array creation? Content of sequences changed (now too shallow).
Exception in thread Thread-12336:
Traceback (most recent call last):
  File "/home/giang/anaconda3/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/home/giang/anaconda3/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-16-2d3616c47f42>", line 5, in detect
RuntimeError: Inconsistent object during array creation? Content of sequences changed (now too shallow).


911
912
913


Exception in thread Thread-12338:
Traceback (most recent call last):
  File "/home/giang/anaconda3/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/home/giang/anaconda3/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-16-2d3616c47f42>", line 5, in detect
RuntimeError: Inconsistent object during array creation? Content of sequences changed (now too shallow).
Exception in thread Thread-12340:
Traceback (most recent call last):
  File "/home/giang/anaconda3/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/home/giang/anaconda3/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-16-2d3616c47f42>", line 5, in detect
RuntimeError: Inconsistent object during array creation? Content of sequences changed (now too shallow).


914
915
916


Exception in thread Thread-12341:
Traceback (most recent call last):
  File "/home/giang/anaconda3/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/home/giang/anaconda3/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-16-2d3616c47f42>", line 5, in detect
RuntimeError: Inconsistent object during array creation? Content of sequences changed (now too shallow).


In [6]:
cv2.destroyAllWindows()

In [7]:
def weighting_func(x):
    return (1 / (1 + np.exp(-0.2 * (x - 16))))
a = []
for i in range(20):
    a.append(weighting_func(i))
a
    

[0.039165722796764356,
 0.04742587317756678,
 0.05732417589886873,
 0.06913842034334682,
 0.08317269649392235,
 0.09975048911968513,
 0.11920292202211755,
 0.14185106490048777,
 0.16798161486607552,
 0.19781611144141822,
 0.23147521650098232,
 0.2689414213699951,
 0.31002551887238755,
 0.35434369377420455,
 0.401312339887548,
 0.45016600268752216,
 0.5,
 0.549833997312478,
 0.598687660112452,
 0.6456563062257954]

In [6]:
event

[250, 420, 1, 427, 1, 434, 1, 442, 1, 452, 1, 465, 1, 481, 1, 501, 1, 525]

In [16]:
a = [4.76569235e-02, 6.91541775e-06 ,6.53933310e-07 ,8.31510180e-07,
 4.23094611e-07 ,3.93461834e-07, 2.25406565e-07, 1.87978748e-07,
 3.85446962e-07 ,3.52457442e-07, 7.62176215e-07 ,4.96316848e-08,
 1.15368251e-06, 7.86701144e-07, 2.32649674e-06, 1.17185538e-07,
 1.31754102e-07 ,4.95291564e-07, 1.56506131e-06 ,2.56187276e-07]
a = np.array(a)
a.sum()

0.047674936375485796

In [8]:
event

[[407, 250, 20],
 [407, 525, 2],
 [407, 652, 2],
 [664, 770, 2],
 [850, 868, 2],
 [873, 899, 2],
 [873, 950, 2],
 [955, 980, 2],
 [955, 1022, 2],
 [1031, 1053, 2],
 [1031, 1341, 3],
 [1031, 1424, 3],
 [1031, 1475, 3]]

In [13]:
cv2.destroyAllWindows()

In [22]:
event.pop(0)

[632, 28, 20]

In [6]:
start_frame

407

In [16]:
end_frame_array

[250, 525, 652, 770, 868, 899, 950, 980, 1022, 1053, 1341]

In [21]:
event

[[632, 28, 20],
 [471, 611, 1],
 [632, 737, 1],
 [754, 852, 1],
 [979, 1000, 2],
 [1041, 1058, 2],
 [1096, 1118, 2],
 [1453, 1473, 3],
 [1500, 1526, 3],
 [1555, 1582, 3],
 [1822, 1863, 4],
 [1892, 1927, 4],
 [1954, 1988, 4],
 [2237, 2259, 5],
 [2290, 2314, 5],
 [2350, 2377, 5],
 [2614, 2643, 6],
 [2674, 2701, 6],
 [2735, 2759, 6],
 [3103, 3146, 7],
 [3157, 3178, 7],
 [3197, 3243, 7],
 [3264, 3308, 7],
 [3517, 3564, 8],
 [3605, 3650, 8],
 [3672, 3684, 8],
 [3697, 3719, 8],
 [4116, 4205, 9],
 [4222, 4277, 9],
 [4298, 4352, 9],
 [4593, 4617, 10],
 [4648, 4678, 10],
 [4710, 4739, 10],
 [5005, 5043, 11],
 [5062, 5103, 11],
 [5125, 5165, 11],
 [5480, 5515, 12],
 [5547, 5585, 12],
 [5616, 5653, 12],
 [5955, 5997, 13],
 [6036, 6072, 13],
 [6108, 6145, 13],
 [6277, 6296, 14],
 [6340, 6359, 14],
 [6403, 6421, 14],
 [6740, 6762, 15],
 [6797, 6821, 15],
 [6860, 6880, 15],
 [6860, 7455, 17],
 [7470, 7487, 17],
 [7470, 7518, 17],
 [7530, 7548, 17],
 [7582, 7614, 17],
 [7993, 8018, 18],
 [8056, 8081,

In [17]:
event.pop(0)
event

[[471, 611, 1], [632, 737, 1]]

In [19]:
# Evaluation phase:
import csv
test_sub = [1,2,3,5,6,7,13,15,17,20]

for i in test_sub:
    csv_file = 'online_mediapipe_{}.csv'.format(i)
    fields = ['Start frame', 'End frame', 'Action']
   
    # 1. step
    with open(csv_file, 'w') as file:

        writer = csv.writer(file)

        writer.writerow(fields)
        file.close()



In [23]:
def write_result_to_csv(event, csv_file):
    for i in event:
# Open our existing CSV file in append mode
# Create a file object for this file
        with open(csv_file, 'a') as f_object:
            writer_object = csv.writer(f_object)
            writer_object.writerow(i)
            f_object.close()

In [25]:
write_result_to_csv(event, 'online_mediapipe_6.csv')

In [28]:
outputs_det

array([0.32535887, 0.67464113], dtype=float32)

In [30]:
a = DD_Net_det.predict(ddnet_det_mp.data_generator_test(sequence1[-16:],C1))[0]

In [31]:
a

array([0.32535887, 0.67464113], dtype=float32)

In [32]:
det_selected_queue

array([0., 0.])

In [34]:
len(sequence1)

32

In [56]:
myqueue_det.size()

76

In [14]:
myqueue_det.size()

31

In [12]:
def detect(DD_Net_det, sequence, ddnet_det_mp, C1):
    
    global outputs_det
    
    outputs_det = DD_Net_det.predict(ddnet_det_mp.data_generator_test(np.array(sequence)[-16:],C1))[0]


    return outputs_det


def classifier(DD_Net_clf, sequence,ddnet_clf_mp,C2 ):
    global outputs_clf
    outputs_clf = DD_Net_clf.predict(ddnet_clf_mp.data_generator_test(np.array(sequence),C2))[0]
    return outputs_clf
    

In [26]:
len(outputs_det)

2

In [22]:
# using raw
import threading
label_wise = []
active_index = 0
passive_count = 0
threshold = 0.6
active = False
num_frame = 0
cum_sum = np.zeros(20, )
clf_selected_queue = np.zeros(20, )
det_selected_queue = np.zeros(2, )
myqueue_det = Queue(6, 2)
clf_threshold_pre = 0.5
myqueue_clf = Queue(16, 20) # clf_queue_size
event = []
start_frame_array = []
end_frame_array = []
label_detected_array = []
det_strategy = 'raw'
clf_strategy = 'ma'
fps_start_time = 0
fps = 0
det_counter = 2
def weighting_func(x):
    return (1 / (1 + np.exp(-0.2 * (x - 12))))
window_size = 32
sequence = []

label = None
event = []
num_frame = 0
prev_frame_time = 0
global res
cap = cv2.VideoCapture('/home/giang/Documents/mica3.webm')
# Set mediapipe model 
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
vid_writer = cv2.VideoWriter('skeleton.avi', fourcc, 18, (width, height))
global res
start_time = time.time()

with mp_pose.Pose(min_detection_confidence=0.45, min_tracking_confidence=0.45) as pose:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()
        frame = cv2.rotate(frame, cv2.ROTATE_180) 
        if not ret:
            print('End of video')
            break
        num_frame = num_frame + 1
        print(num_frame)
        # Make detections
        image, results = mediapipe_detection(frame, pose)


        
        if results.pose_world_landmarks == None:
            print("Frame {} don't have keypoint".format(num_frame))
            label_wise.append(0)
            continue
        # Draw landmarks
        draw_styled_landmarks(image, results)
        # 33x3
        keypoints = extract_keypoints(results)
        # Chuyển thành list
        #keypoints = keypoints.tolist()
        
        
        sequence.append(keypoints)
        
        if len(sequence) == window_size:
            
            t1 = threading.Thread(target=detect, args=(DD_Net_det, sequence, ddnet_det_mp, C1 ,))
            t1.start()
        
            myqueue_det.enqueue(outputs_det.tolist())
            
            # Smooth kết quả đầu ra của bộ detector
            if det_strategy == 'raw':
                det_selected_queue = outputs_det
            elif det_strategy == 'median':
                det_selected_queue = myqueue_det.median
            elif det_strategy == 'ma':
                det_selected_queue = myqueue_det.ma
            elif det_strategy == 'ewma':
                det_selected_queue = myqueue_det.ewma
                
            prediction_det = np.argmax(det_selected_queue)
            
            prob_det = det_selected_queue[prediction_det]
            #print(prob_det)
            
            
            #print(prediction_det) 
            
            # Nếu detect được ứng viên, thực hiện phân loại và hậu xử lý
            if prediction_det == 1:
                
                # CLassifier
                # classifier toàn bộ window
                t2 = threading.Thread(target=classifier, args=(DD_Net_clf, sequence, ddnet_clf_mp,C2 ,))
                t2.start()
                

                # Push the probabilities to queue
                myqueue_clf.enqueue(outputs_clf.tolist())
                
                passive_count = 0
                #print(clf_strategy)
                # Smoothing
                if clf_strategy == 'raw':
                    clf_selected_queue = outputs_clf
                    print(clf_selected_queue)
                elif clf_strategy == 'median':
                    clf_selected_queue = myqueue_clf.median
                elif clf_strategy == 'ma':
                    clf_selected_queue = myqueue_clf.ma
                elif clf_strategy == 'ewma':
                    clf_selected_queue = myqueue_clf.ewma
                
                #print(clf_selected_queue)
                    
                # Nếu ko detect được ứng viên
            else:
                outputs_clf = np.zeros(20, )
                myqueue_clf.enqueue(outputs_clf.tolist())
                
                
                # Đếm số lần ko phát hiện được
                passive_count += 1
                label = None
            
            # Nếu ko detect được ứng viên n lần liên tiếp
            # có active bộ classifier hay không
            #print(passive_count)
            if passive_count > det_counter:
                active = False
                label = None
            else:
                active = True
            # Check passive_count - ok
            #print('passive_count: {} , active : {} '.format(passive_count, active))
            #print(active)
            
            if active:
                active_index +=1
                if active_index > 10 and clf_selected_queue [np.argmax(clf_selected_queue)] >= threshold: 
                    label = 'Action: ' + label_clf[np.argmax(clf_selected_queue)]
                    label_wise.append(np.argmax(clf_selected_queue) +1)
                else:
                    label_wise.append(0)
            

            else:
                label = None
                active_index = 0
                label_wise.append(0)
            # Late detection

            #print ('active: {}, pre_active: {}'.format(active, prev_active))     


            if active == False and prev_active == True:
                pre_predict = False
                event.append(num_frame-6)
                #end_frame_array.append(end_frame)
                #label_detected_array.append(best1+1)
                #event.append([start_frame, end_frame , best1+1])
            prev_active = active
            sequence.pop(0)

            
                                                                        
            # bug here 
        fps_end_time = time.time()
        time_diff = fps_end_time - fps_start_time
        fps = 1/(time_diff)+0.5
        fps_start_time = fps_end_time

        print('========')

            

            
        fps = str(fps)
        fps = fps[0:4]     

        image = cv2.rotate(image, cv2.ROTATE_180)

                         
        #image = cv2.flip(image, 1)
        if label is not None:
            cv2.putText(image, label,
               (220,50), cv2.FONT_HERSHEY_COMPLEX, 0.9, (102, 255, 255), 2)
        cv2.putText(image, 'FPS: {}'.format(fps), (7, 85), cv2.FONT_HERSHEY_COMPLEX, 0.9, (100, 255, 0), 2)
        cv2.putText(image, 'Frame: {}'.format(num_frame), (7, 50), cv2.FONT_HERSHEY_COMPLEX, 0.9, (100, 255, 0), 2)
        # Show to screen
        # Show to screen
        vid_writer.write(image)
        cv2.imshow('Continuous Action Recognition', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30


Exception in thread Thread-18957:
Traceback (most recent call last):
  File "/home/giang/anaconda3/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/home/giang/anaconda3/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-16-2d3616c47f42>", line 5, in detect
RuntimeError: Inconsistent object during array creation? Content of sequences changed (now too shallow).


31
32
33


Exception in thread Thread-18959:
Traceback (most recent call last):
  File "/home/giang/anaconda3/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/home/giang/anaconda3/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-16-2d3616c47f42>", line 5, in detect
RuntimeError: Inconsistent object during array creation? Content of sequences changed (now too shallow).


34
35


...

2896
2897
2898


Exception in thread Thread-23005:
Traceback (most recent call last):
  File "/home/giang/anaconda3/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/home/giang/anaconda3/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-16-2d3616c47f42>", line 5, in detect
RuntimeError: Inconsistent object during array creation? Content of sequences changed (now too shallow).
Exception in thread Thread-23006:
Traceback (most recent call last):
  File "/home/giang/anaconda3/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/home/giang/anaconda3/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-16-2d3616c47f42>", line 5, in detect
RuntimeError: Inconsistent object during array creation? Content of sequences changed (now too shallow).


2899
2900
2901


Exception in thread Thread-23007:
Traceback (most recent call last):
  File "/home/giang/anaconda3/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/home/giang/anaconda3/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-16-2d3616c47f42>", line 5, in detect
RuntimeError: Inconsistent object during array creation? Content of sequences changed (now too shallow).
Exception in thread Thread-23008:
Traceback (most recent call last):
  File "/home/giang/anaconda3/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/home/giang/anaconda3/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-16-2d3616c47f42>", line 5, in detect
RuntimeError: Inconsistent object during array creation? Content of sequences changed (now too shallow).
Exception in thread Thread-23009:
Traceback (most recent call last):
  File "/ho

2902
2903
2904


In [14]:
cv2.destroyAllWindows()

In [17]:
label

In [ ]:
print

In [74]:
myqueue_det.size()

14